This notebook contains tools for examining and managing the processed data files

It includes tasks like: 

- Creating a stopword free database file

- Adding indexes to the database file

- Manually deleting records with non-usable strings

In [ ]:
%cd twitteranalysis
import sqlite3
import environment
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt


#let pandas dataframe listings go long
pd.options.display.max_rows = 999

In [ ]:
from SearchTools.WordMaps import get_adjacent_word_counts, get_adjacent_word_counts_in_tweets, get_user_ids_for_word, get_tweet_ids_for_word
from DataTools import SqliteDataTools as DT

# Properties of master.db

## row counts

In [ ]:
%%time
count_words(environment.USER_DB_NO_STOP)
# environment.USER_DB_NO_STOP
# 11.4 without index
# 6.31 with index

In [ ]:
DT.count_rows(environment.TWEET_DB_MASTER)

In [ ]:
DT.count_rows(environment.TWEET_DB_NO_STOP)

In [ ]:
DT.count_tweets(environment.TWEET_DB_MASTER)

In [ ]:
DT.count_rows(environment.TWEET_DB_NO_STOP)

In [ ]:
DT.count_rows(environment.TWEET_DB_MASTER)

In [ ]:
# Distinct users
DT.count_users()

with exceptions in filter
    - 156590

without 
    - 164499

with regex
    - 144807, 145514

# Maintenance

## Remove bad records

In [ ]:
to_prune = [
    '\\n',
    '\\n\\n',
    '\\xc2\\x96',
    '\\xc3\\xa0',
    '\\xe2\\x80\\x93',
    '\\xe2\\x80\\x94',
    '\\xe2\\x80\\xa6',
    '\\xeb\\x8c\\x80\\xeb\\xb0\\x95',
    '\\xf0\\x9f\\x91\\x8a\\xf0\\x9f\\x8f\\xbb',
    '\\xf0\\x9f\\x98\\x94',
    '\\xf0\\x9f\\x98\\xa2',
    '\\xf0\\x9f\\x98\\xa9',
    '\\xf0\\x9f\\x98\\xab',
    '\\xf0\\x9f\\x98\\xad',
    '\\xf0\\x9f\\x98\\xbc\\xf0\\x9f\\x98\\xb9\\xf0\\x9f\\x99\\x80',
    '/\\xe2\\x80\\xa6',
    'h\\xe2\\x80\\xa6',
    'ht\\xe2\\x80\\xa6',
    'htt\\xe2\\x80\\xa6',
    "'\\xeb\\x8d\\x94\\xeb\\xb8\\x94\\xec\\x97\\x90\\xec\\x8a\\xa4301'\\xec\\x9d\\x84",
    "'\\xeb\\x8d\\x94\\xec\\x87\\xbc",
    "'\\xeb\\x8d\\x94\\xec\\x87\\xbc'\\xec\\x97\\x90\\xec\\x84\\x9c",
    '\xeb\x8d\x94\xec\x87\xbc',
    '\xea\xb9\x80\xea\xb7\x9c\xec\xa2\x85',
    '\xed\x97\x88\xec\x98\x81\xec\x83\x9d'
    '\xea\xb9\x80\xea\xb7\x9c\xec\xa2\x85',
    '//\\xe2\\x80\\xa6',
     '//t.\\xe2\\x80\\xa6',
     '//t.c\\xe2\\x80\\xa6',
     '//t\\xe2\\x80\\xa6',
     '\\xe2\\x80\\x9c',
     '\\xea\\xb9\\x80\\xea\\xb7\\x9c\\xec\\xa2\\x85',
     '\\xea\\xb9\\x80\\xed\\x98\\x95\\xec\\xa4\\x80',
     '\\xeb\\x8d\\x94\\xeb\\xb8\\x94\\xec\\x97\\x90\\xec\\x8a\\xa4301',
     '\\xeb\\x8d\\x94\\xec\\x87\\xbc',
    '\xed\x97\x88\xec\x98\x81\xec\x83\x9d'
]

def prune_map(db):
    conn = sqlite3.connect(db)
    query = "DELETE FROM word_map WHERE word = ?"
    with conn:
        for t in to_prune:
            word = (t, )
            conn.execute(query, word)
            print(u"deleted: %s" % t)

In [ ]:
prune_map(environment.TWEET_DB_NO_STOP)

# Before run: 
# 59756272 rows in /Users/adam/Dropbox/PainNarrativesLab/private_data/tweet-databases/tweets-no-stop.db
# After run:
# 59064116 rows in /Users/adam/Dropbox/PainNarrativesLab/private_data/tweet-databases/tweets-no-stop.db
# Removed 692,156 rows

## Add indexes

In [ ]:
import sqlite3
def add_indexes(db):
    fields = ['user_id', 'tweet_id', 'word']
    query = "create index %s_idx on word_map(%s)"
    conn = sqlite3.connect(db)
    with conn:
        for f in fields:
            q = query % (f, f)
            conn.execute(q)
    
# environment.TWEET_DB_MASTER
# 2.3GB before indexes added
# 5.88 GB after
    

In [ ]:
# add_indexes(environment.TWEET_DB_NO_STOP)

## Create stopword free db

In [ ]:
%cd twitteranalysis
import sqlite3
from nltk.corpus import stopwords
import environment

In [ ]:
def remove_old_table_name(db):
    q = "alter table word_map_deux rename to word_map"
    conn = sqlite3.connect(db)
    conn.execute(q)
    conn.commit()
    conn.close()

In [ ]:
english_stops = stopwords.words('english')

def make_stopword_exclusion_query(stops):
    st = " AND word != '%s'"
    query = "SELECT * FROM word_map WHERE word != '2' "
    for s in stops:
        n = st % s
        query += n 
    return query


def row_generator(source_db, query):
    conn = sqlite3.connect(source_db)
    try:
        r = conn.execute(query)
        while True:
            v = r.fetchone()
            if v is None: raise StopIteration
            yield v
    finally:
        conn.close()




def remove_stopwords(source_db, target_db):
    query = make_stopword_exclusion_query(english_stops)

    conn2 = sqlite3.connect(target_db)

    try:
        userQuery = """INSERT INTO word_map (tweet_id, user_id, word, sentence_index, word_index) VALUES (?, ?, ?, ?, ?)"""
        conn2.executemany( userQuery, row_generator(source_db, query) )
        conn2.commit()
    except StopIteration:
        print('stopped')
    finally:
        conn2.close()
        print('done')



In [ ]:

gen = row_generator(environment.TWEET_DB_MASTER, query)
conn2 = sqlite3.connect(environment.TWEET_DB_NO_STOP)

size = 1000
queue = []

cnt = 0


def save_rows(rows, conn):
#     print('save_rows called; count is %s' % cnt)
    userQuery = """INSERT INTO word_map (tweet_id, user_id, word, sentence_index, word_index) VALUES (?, ?, ?, ?, ?)"""
    conn.executemany( userQuery, rows )
    conn.commit()


prev = 'j'
try:

    while True:    
        cnt += 1
        try:
            if len(queue) >= size:
                save_rows(queue, conn2)
                queue = []
            v = next(gen)
            prev = v
            if v is not None:
                queue.append(v)

        except StopIteration:
            save_rows(queue, conn2)
            
            break

    print('jip')
    
except Exception as e:
    print(e)
    print(prev)
    
finally:
    conn2.close()
    print("count is %s" % cnt)


# Add user ids to the sqlite files to make searching easier

Because we thought leaving them out would save time.... ugh.

In [1]:
%cd twitteranalysis
import environment
import os
import DataTools
import sqlite3
from DataTools.Cursors import WindowedTweetCursor
from DataTools import SqliteDataTools as DT

test_db = '%s/test-ids.db' % environment.LOG_FOLDER_PATH


(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


## Create user id - twitter id map db

In [ ]:

def initialize_id_map_table(db=environment.ID_MAP_DB):
    query = """CREATE TABLE `id_map` (
    `tweet_id` int(20) DEFAULT NULL,
    `user_id` int(20) DEFAULT NULL
    )
  """
    fields = ['user_id', 'tweet_id']
    query2 = "create index %s_idx on id_map(%s)"
    conn = sqlite3.connect(db)
    with conn:
        # create the table
        conn.execute( query )
        conn.commit()
 
        for f in fields:
            q = query2 % (f, f)
            conn.execute(q)
        
        conn.commit()


In [ ]:
os.remove( environment.ID_MAP_DB)

In [ ]:
initialize_id_map_table(environment.ID_MAP_DB)

## Populate the id map table

In [2]:
def populate_id_map(cursor, db=environment.ID_MAP_DB, batch_size=10000):
    query = """insert into id_map(user_id, tweet_id) values(?, ?)"""
    conn = sqlite3.connect(db)
    queue = []
    with conn:
        while True:
            try:
                tweet = cursor.next()
                vals = (tweet.userID, tweet.tweetID)
                queue.append(vals)
                if len(queue) % batch_size == 0:
                    conn.executemany(query, queue)
                    conn.commit()
                    queue = []
            except StopIteration: 
                conn.executemany(query, queue)   
                conn.commit()
                break
            
            

In [ ]:
%%time
cursor = WindowedTweetCursor( language='en' )
populate_id_map(cursor)
# 37 min 12s

## Add them in to the files

This is probably best done by attaching the id map file and then doing a massive select and insert.



In [3]:
gen = DT.master_row_generator()

In [4]:
next(gen)

(331539141059809280, None, 'spoonie', 0, 18)

In [7]:
query = """
UPDATE 
    word_map
SET 
    user_id = (
        SELECT user_id 
        FROM id_map 
        WHERE word_map.tweet_id = id_map.tweet_id
        )
"""

In [8]:
%%time
conn = sqlite3.connect( environment.MASTER_DB )
with conn:
#     curs = conn.cursor()  # Attach cursor
#     query0 = """ATTACH DATABASE '%s' as id_map""" % test_db
    query0 = """ATTACH DATABASE '%s' as id_map""" % environment.ID_MAP_DB
    conn.execute(query0)
    conn.commit()
    conn.execute(query)
    conn.commit()


OperationalError: database is locked

## check whether successful

In [ ]:
# check amount in id_map
# conn = sqlite3.connect( environment.ID_MAP_DB )
conn = sqlite3.connect( test_db )
with conn:
    q = "select count(tweet_id) from id_map where user_id > 0"
    r = conn.execute(q)
    result = r.fetchall()
print(result)

In [9]:
# check number of tweets without user id
conn = sqlite3.connect( environment.MASTER_DB )
with conn:
    q = "select count(tweet_id) from word_map where user_id > 0"
    r = conn.execute(q)
    result = r.fetchall()
print(result)

[(0,)]
